In [ ]:
# Install required packages
!pip install langgraph google-auth google-auth-oauthlib google-auth-httplib2 requests

  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)

  Attempting uninstall: tenacity

    Found existing installation: tenacity 9.1.2

    Uninstalling tenacity-9.1.2:

      Successfully uninstalled tenacity-9.1.2

   ---------------------------------------- 0/4 [tenacity]
   ---------------------------------------- 0/4 [tenacity]
  Attempting uninstall: packaging
   ---------------------------------------- 0/4 [tenacity]
   ---------- ----------------------------- 1/4 [packaging]
    Found existing installation: packaging 25.0
   ---------- ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 0.2.43 which is incompatible.
langchain-text-splitters 0.3.9 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 0.2.43 which is incompatible.
langchain-community 0.2.17 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.27 which is incompatible.
langchain-community 0.2.17 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.3.2 which is incompatible.
marker-pdf 1.8.0 requires Pillow<11.0.0,>=10.1.0, but you have pillow 11.3.0 which is incompatible.
marker-pdf 1.8.0 requires regex<2025.0.0,>=2024.4.28, but you have regex 2025.7.31 which is incompatible.
paddlex 3.0.0 requires numpy==1.24.4; python_version < "3.12", but you have numpy 2.3.2 which is incompatible.
paddlex 3.0.0 requires pand

In [40]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import time
import requests
from google.auth.transport.requests import Request
from google.oauth2 import service_account

In [41]:
def generate_content_with_text(prompt):
    """
    Sends a text prompt to the Gemini model and returns the response and response time.
    """
    start_time = time.time()

    try:
        # Get token from service account
        creds = service_account.Credentials.from_service_account_file(
            "prj-shared-connectivity-rg-001-db7c8a0de3aa.json",
            scopes=["https://www.googleapis.com/auth/cloud-platform"]
        )
        creds.refresh(Request())
        # API details - using the non-streaming endpoint for a complete response
        url = f"https://aiplatform-genai1.p.googleapis.com/v1/projects/prj-shared-connectivity-rg-001/locations/global/publishers/google/models/gemini-2.5-flash:generateContent"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {creds.token}"
        }

        # Request payload (text only)
        data = {
            "contents": [{"role": "user", "parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": 0.4, "topP": 1.0, "topK": 32, "maxOutputTokens": 4000}
        }

        # Make the request
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        
        json_response = response.json()
        response_text = ""
        if "candidates" in json_response and json_response["candidates"]:
            candidate = json_response["candidates"][0]
            if "content" in candidate and "parts" in candidate["content"]:
                for part in candidate["content"]["parts"]:
                    if "text" in part:
                        response_text += part["text"]
        
        response_time = time.time() - start_time
        return {"response": response_text, "response_time": response_time}

    except Exception as e:
        response_time = time.time() - start_time
        error_message = f"An error occurred: {e}"
        if 'response' in locals() and hasattr(response, 'text'):
            error_message += f" | Response content: {response.text}"
        return {"error": error_message, "response_time": response_time}

In [42]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [43]:
def llm_qa(state: LLMState) -> LLMState:
    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question: {question}'

    # ask that question to the Gemini model
    result = generate_content_with_text(prompt)
    
    # check if there was an error
    if "error" in result:
        state['answer'] = f"Error: {result['error']}"
    else:
        state['answer'] = result['response']

    return state

In [44]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [48]:
# execute

intial_state = {'question': 'what is your name AI?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

I do not have a name. I am a large language model, trained by Google.
